# Rule Based Chatbot

In [1]:
import nltk
from nltk.chat.util import Chat, reflections

In [2]:
Chat

nltk.chat.util.Chat

In [3]:
reflections

{'i am': 'you are',
 'i was': 'you were',
 'i': 'you',
 "i'm": 'you are',
 "i'd": 'you would',
 "i've": 'you have',
 "i'll": 'you will',
 'my': 'your',
 'you are': 'I am',
 'you were': 'I was',
 "you've": 'I have',
 "you'll": 'I will',
 'your': 'my',
 'yours': 'mine',
 'you': 'me',
 'me': 'you'}

In [4]:
set_pairs = [
    [
        r"my name is (.*)",["Hello %1, How are you today?", ]
    ], 
    [
        r"hi|hey|hello", ["Hello", "Hey there", ]
    ],
    [
        r"What is your name", ["You can call me a chat bot", ]
    ],
    [
        r"How are you", ["I am fine, thnakyou. How can I help you?",]
    ],
    [
        r"What is your hobby?", ["I keep learning all the time to  upgrade myself",]
    ],
    [
        r"It was nice to converse with you", ["My pleasure!!",]
    ],
    [
        r"quit", ["Bye. see you later!",]
    ],
]

In [5]:
def chatbot():
    print("Hi I am a Rule Based Chat Bot. \nHow can I help you?")
    
#chatbot()

In [6]:
chat = Chat(set_pairs, reflections)

In [7]:
chatbot()
if __name__ == "__main__":
    chat.converse()

Hi I am a Rule Based Chat Bot. 
How can I help you?
>what is your hobby
I keep learning all the time to  upgrade myself
>quit
Bye. see you later!


# Retrieval Based Chatbot

In [1]:
import nltk
import numpy as np
import random
import string

import bs4 as ba
import requests
import re

import warnings
warnings.filterwarnings = False

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
nltk.download('punkt', quiet = True)
nltk.download('wordnet', quiet = True)
nltk.download('popular', quiet=True)

True

In [4]:
req = requests.get(r"https://en.wikipedia.org/wiki/Google_Trends")
raw_html = req.text
req.text

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Google Trends - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"8718f21d-625d-4dfc-9688-a89f950183da","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Google_Trends","wgTitle":"Google Trends","wgCurRevisionId":1023608818,"wgRevisionId":1023608818,"wgArticleId":5101133,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: multiple names: authors list","CS1 German-language sources (de)","Articles with short description","Short description is different from Wikidata","Use

In [5]:
corpus_html = ba.BeautifulSoup(raw_html)
corpus_para = corpus_html.find_all('p')
corpus_text = ''

for para in corpus_para:
    corpus_text = corpus_text + para.text

corpus_text = corpus_text.lower()

In [6]:
corpus_text

'\ngoogle trends is a website by google that analyzes the popularity of top search queries in google search across various regions and languages. the website uses graphs to compare the search volume of different queries over time.\non august 5, 2008, google launched google insights for search, a more sophisticated and advanced service displaying search trends data. on september 27, 2012, google merged google insights for search into google trends.[1]\ngoogle trends also allows the user to compare the relative search volume of searches between two or more terms.\noriginally, google neglected updating google trends on a regular basis. in march 2007, internet bloggers noticed that google had not added new data since november 2006, and trends was updated within a week. google did not update trends from march until july 30, and only after it was blogged about, again.[2] google now claims to be "updating the information provided by google trends daily; hot trends is updated hourly."\non augu

In [7]:
# getting rid all empty spaces and special charectors
corpus_text = re.sub(r'\[[0-9]*\]', ' ', corpus_text)
corpus_text = re.sub(r'\s+', ' ', corpus_text)

# converting text to sentences and word tokens
corpus_senten = nltk.sent_tokenize(corpus_text)
corpus_words = nltk.word_tokenize(corpus_text)

In [8]:
corpus_senten

[' google trends is a website by google that analyzes the popularity of top search queries in google search across various regions and languages.',
 'the website uses graphs to compare the search volume of different queries over time.',
 'on august 5, 2008, google launched google insights for search, a more sophisticated and advanced service displaying search trends data.',
 'on september 27, 2012, google merged google insights for search into google trends.',
 'google trends also allows the user to compare the relative search volume of searches between two or more terms.',
 'originally, google neglected updating google trends on a regular basis.',
 'in march 2007, internet bloggers noticed that google had not added new data since november 2006, and trends was updated within a week.',
 'google did not update trends from march until july 30, and only after it was blogged about, again.',
 'google now claims to be "updating the information provided by google trends daily; hot trends is up

In [39]:
corpus_words

['google',
 'trends',
 'is',
 'a',
 'website',
 'by',
 'google',
 'that',
 'analyzes',
 'the',
 'popularity',
 'of',
 'top',
 'search',
 'queries',
 'in',
 'google',
 'search',
 'across',
 'various',
 'regions',
 'and',
 'languages',
 '.',
 'the',
 'website',
 'uses',
 'graphs',
 'to',
 'compare',
 'the',
 'search',
 'volume',
 'of',
 'different',
 'queries',
 'over',
 'time',
 '.',
 'on',
 'august',
 '5',
 ',',
 '2008',
 ',',
 'google',
 'launched',
 'google',
 'insights',
 'for',
 'search',
 ',',
 'a',
 'more',
 'sophisticated',
 'and',
 'advanced',
 'service',
 'displaying',
 'search',
 'trends',
 'data',
 '.',
 'on',
 'september',
 '27',
 ',',
 '2012',
 ',',
 'google',
 'merged',
 'google',
 'insights',
 'for',
 'search',
 'into',
 'google',
 'trends',
 '.',
 'google',
 'trends',
 'also',
 'allows',
 'the',
 'user',
 'to',
 'compare',
 'the',
 'relative',
 'search',
 'volume',
 'of',
 'searches',
 'between',
 'two',
 'or',
 'more',
 'terms',
 '.',
 'originally',
 ',',
 'google',
 '

In [9]:
wn_lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_corpus(tokens):
    return[wn_lemmatizer.lemmatize(token) for token in tokens]

punct_removal_dict = (dict(ord(punct),None) for punct in string.punctuation)

def get_processed_text(document):
    return lemmatize_corpus(nltk.word_tokenize(document.lower().translate(punct_removal_dict)))

In [12]:
greet_inp = ["Hey", "Hello", "Hola", "Hi", "Greetings", "Helo", "Morning", "Evening"]
greet_out = ["Hi", "Helo", "Hello", "Hey", "Wassup"]

def greet_response(greeting):
    for token in greeting.split():
        if token.split() in greet_inp:
                return(random.choice(greet_out))

In [10]:
#language modeling with Tf-idf
def response(user_query):
    bot_res = ''
    corpus_senten.append(user_query)
    #vectorizing
    word_vectorizer = TfidfVectorizer(tokenizer = get_processed_text, stop_words = 'english')
    corpus_word_vector = word_vectorizer.fit_transform(corpus_senten)
    
    cos_sine_vectors = cosine_similarity(corpus_word_vector[-1], corpus_word_vector)
    similar_response_idx = cos_sine_vectors.argsort()[0][-2]
    
    flat_vect = cos_sine_vectors.flatten()
    flat_vect.sort()
    vect_flat = flat_vect[-2]
    
    if vect_flat == 0:
        bot_res = bot_res+"sorry, cannot understand"
        return bot_res
    else:
        bot_res = bot_res+ corpus_senten[similar_response_idx]
        return bot_res

In [13]:
#conversation
chat = True
print("Hey!! This is your Trend-Bot.\nClarify your queries and get trendy!")
while(chat==True):
    user_query = input()
    user_query = user_query.lower()
    if user_query != 'quit':
        if user_query == 'thanks' or user_query=='Thank you':
            chat = False
            print(" Trend-Bot: You are Welcome")
        else:
            if greet_response(user_query) != None:
                print(" Trend-Bot: "+greet_response(user_query))
            else:
                print(" Trend-Bot: "+response(user_query))
                corpus_senten.remove(user_query)
    else:
        chat = False
        print(" Trend-bot: Good bye!! Take Care!!")

Hey!! This is your Trend-Bot.
Clarify your queries and get trendy!
hi


TypeError: 'generator' object is not subscriptable